In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0685593-3214-4578-a060-6177fc86e747;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable
import pandas as pd

In [5]:
#PATH LANDING ZONE
LANDING_PATH_J_3ABDBS =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher373/sapsr3/pulmaoestoque/eih.bnsvher373.sapsr3.pulmaoestoque.SAPSR3.J_3ABDBS/'
LANDING_PATH_KNA1   =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher373/sapsr3/pulmaoestoque/eih.bnsvher373.sapsr3.pulmaoestoque.SAPSR3.KNA1/'
LANDING_PATH_KNVV   =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher373/sapsr3/pulmaoestoque/eih.bnsvher373.sapsr3.pulmaoestoque.SAPSR3.KNVV/'
LANDING_PATH_TROLZ  =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher373/sapsr3/pulmaoestoque/eih.bnsvher373.sapsr3.pulmaoestoque.SAPSR3.TROLZ/'
LANDING_PATH_TVRO   =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher373/sapsr3/pulmaoestoque/eih.bnsvher373.sapsr3.pulmaoestoque.SAPSR3.TVRO/'

#PATH BRONZE ZONE
BRONZE_PATH_J_3ABDBS =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/pulmaoestoque/sapsr3/pulmaoestoque/j_3abdbs/'
BRONZE_PATH_KNA1    =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/pulmaoestoque/sapsr3/pulmaoestoque/kna1/'
BRONZE_PATH_KNVV    =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/pulmaoestoque/sapsr3/pulmaoestoque/knvv/'
BRONZE_PATH_TROLZ   =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/pulmaoestoque/sapsr3/pulmaoestoque/trolz/'
BRONZE_PATH_TVRO    =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/pulmaoestoque/sapsr3/pulmaoestoque/tvro/'

#PATH SILVER ZONE
SILVER_PATH_J_3ABDBS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/pulmaoestoque/sapsr3/pulmaoestoque/j_3abdbs/'
SILVER_PATH_KNA1 =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/pulmaoestoque/sapsr3/pulmaoestoque/kna1/'
SILVER_PATH_KNVV =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/pulmaoestoque/sapsr3/pulmaoestoque/knvv/'
SILVER_PATH_TROLZ =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/pulmaoestoque/sapsr3/pulmaoestoque/trolz/'
SILVER_PATH_TVRO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/pulmaoestoque/sapsr3/pulmaoestoque/tvro/'


In [6]:
J_3ABDBS = DeltaTable.forPath(spark, SILVER_PATH_J_3ABDBS)

In [7]:
J_3ABDBS_df = J_3ABDBS.toDF()

In [8]:
J_3ABDBS_df_count = J_3ABDBS_df \
    .agg(
        f.count('pk_single').alias('qtd')
    )

In [9]:
J_3ABDBS_df_count.show()

+-------+
|    qtd|
+-------+
|1994706|
+-------+



In [10]:
J_KNA1 = DeltaTable.forPath(spark, SILVER_PATH_KNA1)

In [11]:
J_KNA1_df = J_KNA1.toDF()

In [12]:
J_KNA1_df_count = J_KNA1_df \
    .agg(
        f.count('pk_single').alias('qtd')
    )

In [13]:
J_KNA1_df_count.show()

+------+
|   qtd|
+------+
|113672|
+------+



In [14]:
J_KNA1_df_duplicates = J_KNA1_df \
    .groupBy('pk_single') \
    .agg(
        f.count('pk_single').alias('qtd')
    ) \
    .where(f.col('qtd') > 1) \
    .orderBy('pk_single', ascending = True)

In [15]:
J_KNA1_df_duplicates.show(truncate=False)

+---------+---+
|pk_single|qtd|
+---------+---+
+---------+---+



In [16]:
J_KNA1_df_duplicates_count = J_KNA1_df_duplicates \
    .agg(
        f.count('qtd').alias('qtd_dup')
    ) \
    .where(f.col('qtd_dup') > 1)

In [17]:
J_KNA1_df_duplicates_count.show()

+-------+
|qtd_dup|
+-------+
+-------+



# Duplicates

In [18]:
# j_df_duplicates_example_1 = j_kna1 \
#     .select('timestamp_kafka','KUNNR') \
#     .where(f.col('pk_single') == '0000000017')

In [19]:
# j_df_duplicates_example_1.toPandas()

# Bronze

In [20]:
# J_3ABDBS_b = DeltaTable.forPath(spark, BRONZE_PATH_J_3ABDBS)

In [21]:
# J_3ABDBS_b.toDF().printSchema()

In [22]:
# j_df_b = J_3ABDBS_b.toDF()

In [23]:
# j_df_duplicates_example_2_b = j_df_b \
#     .select('timestamp_kafka','WERKS','MATNR','J_3ASTAT','AUFNR','POSNR','ETENR','BDART','J_3ABSKZ','J_3ABSNR','J_3AHBSP','J_3AEBSP','LGONR','SOBKZ') \
#     .where(f.col('timestamp_kafka') == '2022-07-24 01:53:25.36') \
#     .where(f.col('MATNR') == 'H9981ASI')

In [24]:
# j_df_duplicates_example_2_b.show(truncate=False)